In [1]:
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import pickle
from scipy.spatial import distance_matrix

In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

Credit: A machine learning approach to predicting protein-ligand binding affinity with applications to molecular docking. 
P.J. Ballester & J.B.O. Mitchell, Bioinformatics, 26, 1169-1175 (2010)                                                                                                       
Rewrite of RF-Score_desc.c & RF-Score_pred.r in python, by OZhang 03.2023

Reads a list of PDBbind protein-ligand complexes and calculate their RF-Score descriptors                 

In [3]:
#    1 : ["H" , "HA" , "HA1" , "HA2" , "HB" , "HB1" , "HB2" , "HB3" , "HD1" , "HD2" , 
#         "1HD2", "1HD3", "2HD2", "2HD3", "1HE2", "1HE3", "2HE2", "2HE3",
#         "HE" , "HE1" , "HE2" , "HE3" , "HG" , "HG1" , "HG2" , "1HG2", "1HG3", "2HG2", "2HG3", 
#         "HH", "HH1", "HH2", "1HH1", "2HH1", "1HH2", "2HH2", "HZ1", "HZ2", "HZ3", 
#         "HN1", "HN", "HN2", "HN3"],

In [4]:
NLIGATMAX = 250 # maximum number of heavy atoms in the ligand
NELEMTS = 54   # maximum number of chemical elements considered
DCUTOFF = 12   # distance cutoff for protein atoms near ligand
VERBOSE = 1  # show information about run

atomic_number_to_name = {
    6 : ["C" , "CA" , "CB" , "CD" , "CD1" , "CD2" , "CE" , "CE1" , 
                    "CE2", "CE3", "CG", "CG1", "CG2", "CH2", "CZ", "CZ2", "CZ3"],
    8 : ["O" , "OD1" , "OD2" , "OE1" , "OE1A" , "OE1B" , "OE2" , "OG" , "OG1", "OH", "OXT"],
    7 : ["N" , "NE" , "NE1" , "NE2" , "NE2A" , "NE2B" , "ND1" , "ND2" , "NH1" , "NH2" , "NZ"],
    9 : ["F"],
    15 : ["P"],
    16 : ["S" , "SD" , "SG"],
    17 : ["Cl", "CL"],
    35 : ["Br", "BR"],
    53 : ["I"],
}

name_to_atomic_number = { }
for k, v in atomic_number_to_name.items():
    for i in v:
        name_to_atomic_number[i] = k

In [6]:
def read_pdb_file(filename):
    coords = []
    atomnumbers = []
    with open(filename, "r") as f:
        for line in f.readlines():
            if line.startswith("ATOM"):
                indices = [0, 6, 12, 17, 20, 22, 26, 30, 38, 46, 54, 60, 66, 78]
                items = [line.strip()[i:j] for i, j in zip(indices, indices[1:]+[None])]
                
                if items[-2].strip() == "H":
                    continue
                if name_to_atomic_number.get(items[2].strip()) is None:
                    continue
                    
                atomnumbers.append(name_to_atomic_number.get(items[2].strip()))
                coords += items[7:10]

    assert len(coords)//3 == len(atomnumbers)
    return np.array(atomnumbers), np.reshape(coords, (-1, 3)).astype(float)

def read_ligand_sdf(filename):
    coords = []
    atomnumbers = []
    with open(filename, "r") as f:
        lines = f.readlines()
        natoms = int(lines[3][:3].strip())
        if natoms > NLIGATMAX:
            # then correct reading error
            natoms = int(lines[3][:2].strip())
        for i in range(natoms):
            line = lines[i+4].split()
            atomnumber = name_to_atomic_number.get(line[3])
            if atomnumber is None:
                continue
            atomnumbers.append(atomnumber)
            coords += line[:3]
        assert len(coords)//3 == len(atomnumbers)
        return np.array(atomnumbers), np.reshape(coords, (-1, 3)).astype(float)    

In [7]:
def RF_descriptor(labels, struct_dir):
    #bindaff_dict = read_bindaff(dat_file)
    bindaff_dict = {}
    for prot_name in labels: #bindaff_dict:
        #pname_lc = prot_name.lower()
        ligand_file = os.path.join(struct_dir, prot_name, f"ligand.sdf")
        pocket_file = os.path.join(struct_dir, prot_name, f"protein.pdb")
        if not os.path.exists(ligand_file):
            if VERBOSE: print(prot_name, "does not exists in dir", struct_dir)
            continue

        ligand_a, ligand_c = read_ligand_sdf(ligand_file)
        pocket_a, pocket_c = read_pdb_file(pocket_file)
        features = np.zeros((NELEMTS, NELEMTS))

        # Calculate distances between current ligand and its binding site
        d = distance_matrix(pocket_c, ligand_c)
        dmask = d < DCUTOFF
        lgrid, pgrid = np.meshgrid(ligand_a, pocket_a)
        assert pgrid.shape == dmask.shape
        p_hits = pgrid[dmask]
        l_hits = lgrid[dmask]
        for u in zip(p_hits, l_hits):
            features[int(u[0]), int(u[1])] += 1
            
        col_mask = list(atomic_number_to_name.keys())
        bindaff_dict[prot_name] = features[col_mask][:, col_mask]

    return bindaff_dict

Featurize UCBsplit train data and save as npz

In [ ]:
alldat = pd.read_csv("RF-Score_scripts/ucb_split_info.csv")
f1 = alldat["category"] == "train"
f2 = alldat.CL1 & alldat.CL2
df = alldat[f1 & f2]
feature = RF_descriptor(df["pdbid"], "../ucbsplit/dataset/")
np.savez("./RF-Score_scripts/newtrain_RF_descriptor.npz", **{"pdbid": df["pdbid"],
                                                             "feature": np.array([feature[k] for k in df["pdbid"]]),
                                                             "affinity": df["value"]})

RandomForest training

In [11]:
# Reading Training (TRN) and Test (TST) Datasets; generated with RFscore_descriptor.py
testdata = np.load("RF-Score_scripts/newtest_RF_descriptor.npz")
valdata = np.load("RF-Score_scripts/newval_RF_descriptor.npz")
traindata = np.load("RF-Score_scripts/newtrain_RF_descriptor.npz")

In [12]:
ntrndata = len(traindata["pdbid"])  # number of pdb complexes for training
nvaldata = len(valdata["pdbid"])  
ntstdata = len(testdata["pdbid"])  # number of pdb complexes for testing
seed = 1

itrain = np.arange(ntrndata)
nsample = ntrndata
np.random.seed(seed)
np.random.shuffle(itrain)  # shuffle selected complexes
train_y = np.array(traindata["affinity"])[itrain]
train_X = np.array(traindata["feature"])[itrain]

val_y = np.array(valdata["affinity"])
val_X = np.array(valdata["feature"])

test_y = np.array(testdata["affinity"])
test_X = np.array(testdata["feature"])

In [15]:
# Data pre-processing; remove all zeros entries

col_mask = np.sum(train_X.reshape(-1, 81), axis = 0) > 0
col_mask.sum()

36

In [12]:
#col_mask = np.r_[1:3, 6, 10:12, 15, 19:21, 24, 28:30, 33, 37:39, 42, 
#                 46:48, 51, 55:57, 60, 64:66, 69, 73:75, 78]
train_Xs = train_X[:, col_mask]
test_Xs = test_X[:, col_mask]
val_Xs = val_X[:, col_mask]

In [96]:
# Selecting RF with best internal validation (RF-SCORE)
rmse_OOB_best = 1e8  # dummy high value
for mest in range(100, 550, 50):
    for mtry in range(2, 11):
        RF_mtry = RandomForestRegressor(n_estimators=mest, max_features=mtry, oob_score=True, random_state=24)
        RF_mtry.fit(train_Xs, train_y)
        rmse_OOB = np.sqrt(np.mean((RF_mtry.predict(val_Xs) - val_y) ** 2))
        mae = np.mean((RF_mtry.predict(val_Xs) - val_y))
        if rmse_OOB < rmse_OOB_best:
            mbest = mtry
            rmse_OOB_best = rmse_OOB
            print("mbest = ", mest, mbest, "rmse_OOB = ", round(rmse_OOB, 3))
        #print("mtry = ", mtry)

mbest =  100 2 rmse_OOB =  1.576
mbest =  100 3 rmse_OOB =  1.563
mbest =  100 4 rmse_OOB =  1.55


In [97]:
test_rmse = []
corr_avg = np.zeros((10, 2))
best_fit = None
best_model = None
for n in range(10):
    RF_Score = RandomForestRegressor(n_estimators=100, max_features=3)
    RF_Score.fit(train_Xs, train_y)

    # train performance
    train_pred = RF_Score.predict(train_Xs)
    train_rmse = np.round(((train_y - train_pred)**2).mean() ** 0.5, 3)
    train_sdev = np.round((train_y - train_pred).std(), 3)
    fitpoints = pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_pred)], axis=1).dropna()
    tr_fitcorr = np.round(fitpoints.corr(method='pearson').iloc[0, 1], 3)
    tr_sprcorr = np.round(fitpoints.corr(method='spearman').iloc[0, 1], 3)
    print("pearson R", tr_fitcorr, "spearman R", tr_sprcorr)
    
    # test performance
    test_pred = RF_Score.predict(test_Xs)
    test_rmse.append(np.round(((test_y - test_pred)**2).mean() ** 0.5, 3))
    #test_sdev = np.round((test_y - test_pred).std(), 3)
    fitpoints = pd.concat([pd.DataFrame(test_y), pd.DataFrame(test_pred)], axis=1).dropna()
    fitcorr = np.round(fitpoints.corr(method='pearson').iloc[0, 1], 3)
    sprcorr = np.round(fitpoints.corr(method='spearman').iloc[0, 1], 3)
    print(f"run {n+1}: rmse {test_rmse[-1]}; pearson corr {fitcorr}; spearman corr {sprcorr}")
    corr_avg[n] = np.array([fitcorr, sprcorr])
    if fitcorr == np.max(corr_avg[:, 0]):
        best_fit = fitpoints
        best_model = RF_Score

pearson R 0.976 spearman R 0.976
run 1: rmse 1.55; pearson corr 0.564; spearman corr 0.531
pearson R 0.976 spearman R 0.975
run 2: rmse 1.576; pearson corr 0.548; spearman corr 0.516
pearson R 0.976 spearman R 0.976
run 3: rmse 1.551; pearson corr 0.565; spearman corr 0.535
pearson R 0.976 spearman R 0.976
run 4: rmse 1.58; pearson corr 0.546; spearman corr 0.511
pearson R 0.976 spearman R 0.976
run 5: rmse 1.567; pearson corr 0.553; spearman corr 0.521
pearson R 0.976 spearman R 0.975
run 6: rmse 1.55; pearson corr 0.563; spearman corr 0.531
pearson R 0.976 spearman R 0.975
run 7: rmse 1.562; pearson corr 0.556; spearman corr 0.525
pearson R 0.976 spearman R 0.975
run 8: rmse 1.568; pearson corr 0.552; spearman corr 0.518
pearson R 0.976 spearman R 0.976
run 9: rmse 1.55; pearson corr 0.566; spearman corr 0.533
pearson R 0.976 spearman R 0.976
run 10: rmse 1.558; pearson corr 0.558; spearman corr 0.527


In [98]:
# save models
with open('RF_ucbsplit.pkl', 'wb') as handle:
    pickle.dump(best_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

Compare models performance

In [10]:
with open('RF_pdb2007.pkl', 'rb') as f:
    old_model = pickle.load(f)

with open('RF_ucbsplit.pkl', 'rb') as f:
    best_model = pickle.load(f)

In [16]:
df = pd.read_csv("../BindingDB_test/bindingDB_processed.csv")
feature = RF_descriptor(df["PDBID"], "../BindingDB_test/dataset/")
DB_Xs = np.reshape(feature, (-1, 81))
DB_Xs = DB_Xs[:, col_mask]
old_pred = old_model.predict(DB_Xs)
new_pred = best_model.predict(DB_Xs)